<a href="https://colab.research.google.com/github/soulsharp/Triton_kernels_ViT/blob/main/Triton_Softmax_Stable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.9 MB/s eta 0:00:00


In [3]:
import torch
import triton
import triton.language as tl

**APPROACH 1:**

Multiple loads from global memory, but no pressure on shared memory



In [4]:
@triton.jit
def _softmax_stable_blocked(
    input_ptr,
    output_ptr,
    input_row_stride,
    output_row_stride,
    num_cols,
    BLOCK_SIZE: tl.constexpr,
):

    pid = tl.program_id(0)
    input_row = input_ptr + pid * input_row_stride
    output_row = output_ptr + pid * output_row_stride

    # Shared memory arrays for partial reductions
    num_blocks = tl.cdiv(num_cols, BLOCK_SIZE)
    partial_max = tl.zeros([num_blocks], dtype=tl.float32)
    partial_sum = tl.zeros([num_blocks], dtype=tl.float32)

    # Computes the row max in blocks
    for block_id in range(num_blocks):
        start_idx = block_id * BLOCK_SIZE
        offsets = start_idx + tl.arange(0, BLOCK_SIZE)
        mask = offsets < num_cols
        values = tl.load(input_row + offsets, mask=mask, other=0)

        # Gets the max of the elements in the block(partial max)
        partial_max[block_id] = tl.max(values, axis=0)

    # Reduces partial maxima to find the global maximum
    row_max = tl.max(partial_max, axis=0)

    # Computes the exponentials and partial sums
    for block_id in range(num_blocks):
        start_idx = block_id * BLOCK_SIZE
        offsets = start_idx + tl.arange(0, BLOCK_SIZE)
        mask = offsets < num_cols

        values = tl.load(input_row + offsets, mask=mask, other=-float('inf'))

        # Subtracts row max and exponentiates
        values = tl.exp(values - row_max)

        # Gets the partial sum of the elements in the  block
        partial_sum[block_id] = tl.sum(values, axis=0)

        # Writes back intermediate values for normalization
        tl.store(output_row + offsets, values, mask=mask)

    # Reduces partial sums to find the global sum
    row_sum = tl.sum(partial_sum, axis=0)

    # Normalizes and writes back the final softmax values
    for block_id in range(num_blocks):
        start_idx = block_id * BLOCK_SIZE
        offsets = start_idx + tl.arange(0, BLOCK_SIZE)
        mask = offsets < num_cols

        # Loads partially normalized values(x-mean)
        values = tl.load(output_row + offsets, mask=mask, other=0)
        values /= row_sum

        # Writes back the final normalized values
        tl.store(output_row + offsets, values, mask=mask)


**APPROACH 2 :**

Loads each row of the matrix once and performs all computations using
shared memory or registers.

Becomes shared memory-bound when the size of rows is large.

In [5]:
@triton.jit
def _softmax_stable(
    input_ptr,
    output_ptr,
    input_row_stride,
    output_row_stride,
    num_cols,
    BLOCK_SIZE:tl.constexpr,
):

  pid = tl.program_id(0)
  input_row = input_ptr + pid * input_row_stride
  partial_max = tl.zeros([tl.cdiv(num_cols, BLOCK_SIZE)], dtype=tl.float32)
  partial_sum = tl.zeros([tl.cdiv(num_cols, BLOCK_SIZE)], dtype=tl.float32)
  shared_mem_row = tl.zeros([num_cols], dtype=tl.float32)
  partial_max_mem_idx = 0
  partial_sum_mem_idx = 0

  for offset in range(0, num_cols, BLOCK_SIZE):
    offsets = offset + tl.arange(0, BLOCK_SIZE)
    mask = offsets < num_cols

    # row_segment = tl.load(input_row + offsets, mask=mask)
    shared_mem_row[offsets] = tl.load(input_row + offsets, mask=mask)
    partial_max[partial_max_mem_idx] = tl.max(shared_mem_row, axis=0)
    partial_max_mem_idx += 1

  row_max = tl.max(partial_max, axis=0)

  for offset in range(0, num_cols, BLOCK_SIZE):
    offsets = offset + tl.arange(0, BLOCK_SIZE)
    mask = offsets < num_cols

    # row_segment = tl.load(input_row + offsets, mask=mask)
    shared_mem_row[offsets] = shared_mem_row[offsets] - row_max
    shared_mem_row[offsets] = tl.exp(shared_mem_row[offsets])
    partial_sum[partial_sum_mem_idx] = tl.sum(shared_mem_row[offsets], axis=0)
    partial_sum_mem_idx += 1

  row_sum = tl.sum(partial_sum, axis=0)

  for offset in range(0, num_cols, BLOCK_SIZE):
    offsets = offset + tl.arange(0, BLOCK_SIZE)
    mask = offsets < num_cols

    shared_mem_row[offsets] /= row_sum

    tl.store[output_ptr + offsets, shared_mem_row[offsets], mask]

In [6]:
@triton.jit
def _softmax_backward_kernel(
    input_ptr,  # ptr to inputs of softmax (Y values)
    dldy_ptr,   # DL/DY (derivative of loss wrt softmax outputs)
    output_ptr, # DL/DX (derivative of loss wrt logits)
    input_row_stride,
    output_row_stride,
    num_cols,
    BLOCK_SIZE: tl.constexpr,
):
    pid = tl.program_id(0)
    input_row = input_ptr + pid * input_row_stride
    output_row = output_ptr + pid * output_row_stride
    dldy_row = dldy_ptr + pid * input_row_stride
    num_blocks = tl.cdiv(num_cols, BLOCK_SIZE)

    # Block-wise partial dot product sums
    block_dot_sum = tl.zeros([num_blocks], dtype=tl.float32)

    # Computes partial dot products
    for block_id in range(num_blocks):
        start_idx = block_id * BLOCK_SIZE
        offsets = start_idx + tl.arange(0, BLOCK_SIZE)
        mask = offsets < num_cols

        # Loads softmax outputs (Y) and DL/DY
        y_values = tl.load(input_row + offsets, mask=mask)
        dldy_values = tl.load(dldy_row + offsets, mask=mask)

        # Accumulates partial dot product
        block_dot_sum[block_id] = tl.sum(y_values * dldy_values, axis=0)

    # Reduces dot product across all blocks
    final_dot_sum = tl.sum(block_dot_sum, axis=0)

    # Computes DL/DX for each block
    for block_id in range(num_blocks):
        start_idx = block_id * BLOCK_SIZE
        offsets = start_idx + tl.arange(0, BLOCK_SIZE)
        mask = offsets < num_cols

        # Loads softmax outputs (Y) and DL/DY
        y_values = tl.load(input_row + offsets, mask=mask)
        dldy_values = tl.load(dldy_row + offsets, mask=mask)

        # Computes DL/DX gradient and stores them back
        dldx_grad = y_values * (dldy_values - final_dot_sum)
        tl.store(output_row + offsets, dldx_grad, mask=mask)

